# Kaggle: Predict Loan Payback — Feature Engineering

**Notebook:** `03_feature_engineering.ipynb`
**Author:** Brice Nelson
**Organization:** Kaggle Series | Brice Machine Learning Projects
**Date Created:** November 16, 2025
**Last Updated:** November 16, 2025

---

## 🧭 Purpose

This notebook focuses on **feature engineering** for the *Predict Loan Payback* Kaggle competition.
Feature engineering is where we convert domain knowledge and statistical insight into **predictive power**, increasing the model’s ability to distinguish between loans that are repaid vs. defaulted.

Whereas `02_data_cleaning.ipynb` prepared a clean, numeric, and aligned dataset, this notebook builds **new, informative features** that enhance model signal.

### **Objectives**
1. Import scaled training and test datasets from `/data/processed/`.
2. Engineer domain-driven features (credit, income, debt, and loan characteristics).
3. Validate distributions and inspect transformations.
4. Export a feature-enhanced dataset for modeling.

---

## 🧱 Feature Engineering Roadmap

The engineered features planned for this notebook:

### **1. Financial Ratio Features**
- **Loan-to-Income Ratio:** `loan_amount / annual_income`
- **Debt-to-Income Bucket:** Flag or bin DTI levels
- **Interest Rate Stress Feature:** Higher-than-expected interest rates relative to credit score

### **2. Credit Behavior Features**
- **Credit Score Buckets:** Very low → very high
- **Grade/Subgrade Interactions:** E.g., `grade * subgrade`
- **Creditworthiness Flags:** Threshold-based indicators

### **3. Loan Purpose & Behavior Features**
- One-hot interactions (e.g., `purpose × credit_score_bucket`)
- Risk grouping based on purpose type

### **4. Optional Modeling-Useful Transforms**
- Log transforms for skewed features
- Quantile transforms (if needed)

---

## 📥 Load Processed Data

The processed datasets created in `02_data_cleaning.ipynb` will now be imported from:
`../data/processed/loan_train_scaled.csv` and `../data/processed/loan_test_scaled.csv`.

